In [58]:
import os
import cv2
import json
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import fiftyone as fo
import fiftyone.zoo as foz
from pycocotools.coco import COCO
import numpy as np

#dataset = foz.load_zoo_dataset("coco-2017")

In [70]:
def load_coco_data(images_dir, ann_file_caption, ann_file_bbox, withCaption=True):
    coco_caption = COCO(ann_file_caption)
    coco_bbox = COCO(ann_file_bbox)
    image_ids = list(coco_caption.imgs.keys())
    
    def load_image_and_annotations(image_id):
        img_info = coco_caption.loadImgs(image_id)[0]
        img_path = os.path.join(images_dir, img_info['file_name'])
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)
        
        ann_ids_caption = coco_caption.getAnnIds(imgIds=image_id)
        anns_caption = coco_caption.loadAnns(ann_ids_caption)
        captions = [ann['caption'] for ann in anns_caption if 'caption' in ann]

        ann_ids_bbox = coco_bbox.getAnnIds(imgIds=image_id);
        anns_bbox = coco_bbox.loadAnns(ann_ids_bbox)
        bboxes = [ann['bbox'] for ann in anns_bbox if 'bbox' in ann]
        
        if withCaption:
            return img, ann_ids_caption, ann_ids_bbox, captions[0] if captions else "<no caption>", bboxes
        else:
            return img, ann_ids_bbox, bboxes

    return image_ids, load_image_and_annotations

In [71]:
coco_images_dir = "/Users/ivanng/fiftyone/coco-2017/train/data"
coco_ann_file_caption = "/Users/ivanng/fiftyone/coco-2017/raw/captions_train2017.json"
coco_ann_file_bbox = "/Users/ivanng/fiftyone/coco-2017/raw/instances_train2017.json"

image_ids, load_image_and_annotations = load_coco_data(coco_images_dir, coco_ann_file_caption, coco_ann_file_bbox, caption_mode=True)

# Iterate through a subset of images and examine content
for image_id in image_ids[:20]:  # Limit to first 5 images for inspection
    img, captionid, bboxid, caption, bboxes = load_image_and_annotations(image_id)
    print(f"Image ID: {image_id}")
    print(f"Caption ID: {captionid}")
    print(f"bbox ID: {bboxid}")
    print(f"Image shape: {img.shape}")
    print(f"Caption: {caption}")
    print(f"Bounding boxes: {bboxes}")
    print('-' * 50)

loading annotations into memory...
Done (t=0.75s)
creating index...
index created!
loading annotations into memory...
Done (t=27.79s)
creating index...
index created!
Image ID: 391895
Caption ID: [770337, 771687, 772707, 776154, 781998]
bbox ID: [151091, 202758, 1260346, 1766676]
Image shape: (360, 640, 3)
Caption: A man with a red helmet on a small moped on a dirt road. 
Bounding boxes: [[359.17, 146.17, 112.45, 213.57], [339.88, 22.16, 153.88, 300.73], [471.64, 172.82, 35.92, 48.1], [486.01, 183.31, 30.63, 34.98]]
--------------------------------------------------
Image ID: 522418
Caption ID: [681330, 686718, 688839, 693159, 693204]
bbox ID: [455475, 692513, 1085508, 1982455]
Image shape: (480, 640, 3)
Caption: A woman wearing a net on her head cutting a cake. 
Bounding boxes: [[382.48, 0.0, 256.8, 474.31], [234.06, 406.61, 219.94, 42.67], [0.0, 316.04, 406.65, 157.49], [305.45, 172.05, 57.36, 77.3]]
--------------------------------------------------
Image ID: 184613
Caption ID: [474

In [ ]:
# overall function
#capture = cv2.VideoCapture(video_path):
#frameCount = 0;
#while capture.isOpened():
    #ret, frame = capture.read();
    #if not ret:
        #print('Video stream end')
        #break
    #frameCount+=1
    # Classifier
    